In [2]:
import tensorflow as tf
import os # para manipular archivos
import cv2
import numpy as np


In [19]:
base_dir = 'asl_data_set'

In [8]:
# preprocesamiento de los datos, es decir, preparar los datos en crudo para construir y entrenar modelos
IMAGE_SIZE = 224 # tamaño de las imagenes 
BATCH_SIZE = 64  # cantidad de images que se van a procesar en cada paso

# preprocsamiento de la imagen
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    # rescale = 1./255, # reescalamos las imagenes
    validation_split=0.2 # separar los datos en 2 grupos, uno para entrenar y otro para validar
    # de manera convencional, el 80% de los datos se usan para entrenar y el 20% para validar
)

# generador de datos de entrenamiento
train_generator = datagen.flow_from_directory( 
    base_dir, # directorio base que contiene las carpetas de las imagenes
    target_size=(IMAGE_SIZE, IMAGE_SIZE),  # convertir las imagenes a un tamaño uniforme de 224x224
    batch_size = BATCH_SIZE, # las imagenes se procesan en lotes de 64 en la red neuronal
    subset='training' # el nombre que asignamos a la carpeta de entrenamiento
)

# generador de datos de validacion
val_generator = datagen.flow_from_directory(  
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

NameError: name 'base_dir' is not defined

In [6]:
# A continuacion, tenemos que crear un archivo de etiquetas que contendra todas nuestras etiquetas (importante para Flutter)
print(train_generator.class_indices) # imprime las etiquetas de las imagenes del dataset
labels = '\n'.join(sorted(train_generator.class_indices.keys())) # imprime las keys como etiquetas en un archivo de texto llamado labels.txt
with open('labels.txt', 'w') as f: # escribe en el archivo labels.txt, y si no existe, lo crea, y si existe, lo sobreescribe. (eso es lo que 'w' es para)
    f.write(labels)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'space': 27}


In [7]:
# construimos una red neuronal usando el metodo de transferencia de aprendizaje donde tomamos una red neuronal preentrenada llamada MobileNetV2
#  que es una arquitectura de red neuronal convolucional que busca funcionar bien en dispositivos moviles y puede predecir hasta 80 clases diferentes
# tendremos una red neuronal base en la parte superior de la cual agregaremos una red neuronal preentrenada para que prediga las clases que queremos
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3) 
base_model = tf.keras.applications.MobileNetV2( # MobileNetV2 es una red neuronal convolucional preentrenada
    input_shape=IMG_SHAPE,
    include_top=False, # esto parara los pesos de la red neuronal en la parte superior, porque no queremos que se entrenen, porque ya estan entrenados
    weights='imagenet'
)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-11-16 16:30:30.716604: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-16 16:30:30.717125: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


9406464/9406464 [==============================] - 1s 0us/step


In [8]:
base_model.trainable=False # esto congelara todos los neuronas para nuestro modelo base
model = tf.keras.Sequential([ # redes neuronales actuan en una secuencia de capas, asi que agregamos capas a medida que lo necesitamos
  base_model,
  tf.keras.layers.Conv2D(32,3, activation = 'relu'), # esta capa crea un kernel de convolucion que se convoluciona con la entrada de la capa para producir una tensor de salida
  tf.keras.layers.Dropout(0.2), # esta capa evita el sobreajuste, es decir, que la red neuronal sea demasiado precisa hasta el punto de que solo pueda reconocer imagenes que estan presentes en el conjunto de datos
  tf.keras.layers.GlobalAveragePooling2D(), # esta capa calcula el promedio de salida de cada mapa de caracteristicas en la capa anterior, lo que reduce los datos significativamente y prepara el modelo para la capa final
  tf.keras.layers.Dense(28, # numero de clases que queremos predecir
                        activation='softmax')
])

In [9]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(), # Adam es un optimizador popular, diseñado específicamente para el entrenamiento de redes neuronales profundas
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [10]:
epochs = 10 # a mayor numero de epochs, mas preciso sera el modelo, pero tambien puede causar sobreajuste, si es demasiado alto
history = model.fit(
    train_generator, 
    epochs = epochs, 
    validation_data=val_generator
)

Epoch 1/10


2022-11-16 16:33:43.692655: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-16 16:33:44.545767: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1050/1050 [==============================] - ETA: 0s - loss: 1.8301 - accuracy: 0.4410

2022-11-16 16:35:35.155524: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1050/1050 [==============================] - 137s 126ms/step - loss: 1.8301 - accuracy: 0.4410 - val_loss: 1.8442 - val_accuracy: 0.4216
Epoch 2/10
1050/1050 [==============================] - 131s 125ms/step - loss: 0.9927 - accuracy: 0.6804 - val_loss: 1.6681 - val_accuracy: 0.4569
Epoch 3/10
1050/1050 [==============================] - 130s 124ms/step - loss: 0.8139 - accuracy: 0.7342 - val_loss: 1.4752 - val_accuracy: 0.5232
Epoch 4/10
1050/1050 [==============================] - 132s 126ms/step - loss: 0.7165 - accuracy: 0.7652 - val_loss: 1.6163 - val_accuracy: 0.5033
Epoch 5/10
1050/1050 [==============================] - 131s 124ms/step - loss: 0.6450 - accuracy: 0.7860 - val_loss: 1.6275 - val_accuracy: 0.5104
Epoch 6/10
1050/1050 [==============================] - 130s 124ms/step - loss: 0.5937 - accuracy: 0.8035 - val_loss: 1.4543 - val_accuracy: 0.5464
Epoch 7/10
1050/1050 [==============================] - 130s 123ms/step - loss: 0.5535 - accuracy: 0.8172 - val_loss: 1.531

In [11]:
# con nuestra red neuronal entrenada con tensorflow y keras, podemos exportarla
saved_model_dir = '' # directorio donde se guardara el modelo
tf.saved_model.save(model, saved_model_dir) # guarda el modelo en el directorio

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) 
tflite_model = converter.convert() # convierte el modelo en un modelo de tensorflow lite, que pueda usar flutter 
with open('model.tflite', 'wb') as f: # guardamos el modelo en un archivo llamado model.tflite, y si no existe, lo crea, y si existe, lo sobreescribe. (eso es lo que 'wb' es para)
  f.write(tflite_model)

INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: /var/folders/w9/47hfrj5s20n22dxytnnn2mjh0000gn/T/tmpd0f_nov3/assets


INFO:tensorflow:Assets written to: /var/folders/w9/47hfrj5s20n22dxytnnn2mjh0000gn/T/tmpd0f_nov3/assets
2022-11-16 16:59:42.026766: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-16 16:59:42.026921: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-16 16:59:42.029963: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/w9/47hfrj5s20n22dxytnnn2mjh0000gn/T/tmpd0f_nov3
2022-11-16 16:59:42.049095: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-11-16 16:59:42.049111: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/w9/47hfrj5s20n22dxytnnn2mjh0000gn/T/tmpd0f_nov3
2022-11-16 16:59:42.091948: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-11-16 16:59:42.106683: I tensorflow/cc/saved_model/load

In [6]:
# Cargamos el modelo en memoria 
path = 'model.tflite' # ruta del modelo
interpreter = tf.lite.Interpreter(model_path=path) 
interpreter.allocate_tensors()

In [11]:
# cargamos la imagen
path = 'asl_test/M_test.jpg'
img = tf.keras.preprocessing.image.load_img(path , target_size=(IMAGE_SIZE, IMAGE_SIZE)) # cargamos la imagen de prueba
img_array = tf.keras.preprocessing.image.img_to_array(img) # convertimos la imagen a un array
img_array = tf.expand_dims(img_array, 0) # expandimos las dimensiones del array


# predecimos la imagen 
input_details = interpreter.get_input_details() # obtenemos los detalles de la entrada
output_details = interpreter.get_output_details() # obtenemos los detalles de la salida

# convertimos a float32
input_shape = input_details[0]['shape']
input_data = np.array(img_array, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke() # invocamos el modelo
output_data = interpreter.get_tensor(output_details[0]['index']) # obtenemos la salida
# print(output_data) # imprimimos la salida

# obtenemos la etiqueta de la prediccion
with open('labels.txt', 'r') as f:
    labels = [line.strip() for line in f.readlines()]
prediction = np.argmax(output_data) # obtenemos el indice de la prediccion
print(labels[prediction]) # imprimimos la etiqueta de la prediccion


M
